In [9]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymysql

ModuleNotFoundError: No module named 'pymysql'

#### 모든 제품을 크롤링해서 DB에 저장
- 저장할 DB부터 테이블 생성해서 형식에 맞게 data 변환 후 DB로 insert
1. DB 연결 코드
2. 필요한 DB 및 table 생성
3. 크롤링
    - 데이터를 table 형식에 맞게 정제
4. insert (데이터 생성)
5. commit() 해서 db에 반영
6. db 닫기

- db 테이블을 읽어와서 df 에 저장

1. DB연결 및 필요 객체 생성

In [17]:
def conn(d_name) :
    import pymysql
    host_name = 'localhost'
    host_port = 3306
    username = 'root'
    password = '0000'
    database_name = d_name
    db = pymysql.connect(
        host=host_name,     # MySQL Server Address
        port=host_port,          # MySQL Server Port
        user=username,      # MySQL username
        passwd=password,    # password for MySQL username
        db=database_name,   # Database name
        charset='utf8'
    )
    return db

In [18]:
db = conn('none')
cursor = db.cursor()

In [19]:
sql = 'create database beauty_shop'
cursor.execute(sql)

1

In [20]:
sql = "show databases"
cursor.execute(sql)
result = cursor.fetchall()
result

(('beauty_shop',),
 ('information_schema',),
 ('mysql',),
 ('none',),
 ('performance_schema',),
 ('test',))

In [21]:
sql = "use beauty_shop"
cursor.execute(sql)

0

In [22]:
sql = "select database()"
cursor.execute(sql)
result = cursor.fetchone()
result

('beauty_shop',)

In [23]:
# table 생성
sql = '''
    CREATE TABLE product (
        PRODUCT_CODE int AUTO_INCREMENT NOT NULL,
        TITLE VARCHAR(200) NOT NULL,
        ORI_PRICE FLOAT,
        DISCOUNT_PRICE FLOAT,
        link VARCHAR(200),
        PRIMARY KEY(PRODUCT_CODE)
    );
'''

cursor.execute(sql)
db.commit()

In [24]:
sql = "show tables"
cursor.execute(sql)
result = cursor.fetchall()
result

(('product',),)

In [25]:
sql = 'desc product'
cursor.execute(sql)
result = cursor.fetchall()
result

(('PRODUCT_CODE', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('TITLE', 'varchar(200)', 'NO', '', None, ''),
 ('ORI_PRICE', 'float', 'YES', '', None, ''),
 ('DISCOUNT_PRICE', 'float', 'YES', '', None, ''),
 ('link', 'varchar(200)', 'YES', '', None, ''))

2. 크롤링 코드 - insert 구문을 추가해서 변경

In [26]:
# 크롤링 문서 요청해서 응답객체 반환(첫번째 크롤링 문서 요청 후 응답)
url="http://jolse.com/category/toners-mists/1019/"
html = urlopen(url)
htmls = html.read()
# print(htmls)
bs_obj = BeautifulSoup(htmls,"html.parser")

In [27]:
# box안에 들어 있는 1개의 상품에서 정보를 추출해서 dict형태로  반환하는 함수
# 데이터 전처리 : 제품명에 ' 제거/ 가격 USD 제거 / 세일가격 없는 경우 처리
def get_product_info(box) :
    p_tag = box.find("p",{"class":"name"})
    # 품목 추출
    span = p_tag.find("span")
    
    # 세부페이지링크 추출
    a = p_tag.find("a")
    sub_link = 'https://jolse.com' + a["href"]
    # 가격 추출 코드
    price_ul = box.find("ul")
    price_span = price_ul.findAll("span")
    
    # 데이터 전처리
    title = span.text.replace("'","''") # ' 처리
    ord_price = price_span[1].text.split(' ')[1] # USD 제거
    dis_price = price_span[-1].text.split(' ')[1]
    # 세일 가격이 없는 경우 
    if dis_price =='' :
        dis_price = '0.0'
    
    
    # 최종 data 추출 후 반환 
    return{"prd_name":title,"price":ord_price,"sale_price":dis_price,"sub_link":sub_link}


In [28]:
def save_data(prd_info) :
#     print(prd_info)
    
    # insert 구문
    sql = "INSERT INTO product (title, ori_price, discount_price,link) values('" \
        + prd_info["prd_name"] \
        +"'," \
        + prd_info['price']\
        +"," \
        + prd_info['sale_price']\
        +",'"\
        + prd_info['sub_link']\
        + "')"
#     print(sql)
    cursor.execute(sql)

In [29]:
# 전달된 url 페이지에 접근해서 해당페지의 전체 상품 데이터를 추출 한 후 각 상품마다 get_product_info()함수를 호출해서
# 각 상품에대한 추출 정보를 받아옴 - 들어온 각 상품 정보를 리스트에 저장 한 수 해당 반환
def get_page_products(url) :
    url=url
    html = urlopen(url)
    htmls = html.read()
    # print(htmls)
    bs_obj = BeautifulSoup(htmls,"html.parser")

    ## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
    # ul class:prdList grid4
    ul=bs_obj.find("ul",{"class":"prdList grid4"})
    ## 품목 1개를 담고 있는 div 태그 추출
    ## div class:box
    prd_boxes = ul.findAll("div", {"class":"box"}) #1개 페이지의 전체 상품
    # 반환되는 품목 데이터를 db에 insert  : 함수호출해서 진행
    for box in prd_boxes :
        prd = get_product_info(box)
#         print(prd)
        save_data(prd) #사용자 정의 함수(생성해야 함)

### main 코드(프로그램 시작점)

In [30]:
from tqdm import tqdm_notebook # 상태바 표시

#여러 페이지의 화장품 정보를 추출해서 df 에 저장 후 csv에 저장하는 코드
url = "http://jolse.com/category/toners-mists/1019/?page=" #페이지 번호를 제외한 공통 url 문자열

last= int(bs_obj.find("p",{"class":"last"}).find("a")['href'].split("=")[1])

for i in tqdm_notebook(range(2,last+1)) : # 2페이지부터 수집
# for i in range(1,2) : # 1페이지 insert
    # url 완성 :page번호를 추가 i 변수 값을 활용
    urlfin =url + str(i)
    get_page_products(urlfin)


<ipython-input-30-7519439ecfd9>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(2,last+1)) : # 2페이지부터 수집


  0%|          | 0/11 [00:00<?, ?it/s]

In [31]:
db.commit()

In [32]:
sql = "select * from product"
cursor.execute(sql)
result = cursor.fetchall()
result

((1,
  'B_LAB Matcha Hydrating Facial Toner 150ml',
  17.0,
  11.9,
  'https://jolse.com/product/detail.html?product_no=42845&cate_no=1019&display_group=1'),
 (2,
  'META FORET 12PM Calming Toner 150ml',
  13.2,
  11.22,
  'https://jolse.com/product/detail.html?product_no=42825&cate_no=1019&display_group=1'),
 (3,
  'LOOKS&MEII Rich Routine Balance Toner 150ml',
  21.0,
  17.85,
  'https://jolse.com/product/detail.html?product_no=42564&cate_no=1019&display_group=1'),
 (4,
  'LOOKS&MEII Fresh Routine Balance Toner 150ml',
  21.0,
  17.85,
  'https://jolse.com/product/detail.html?product_no=42547&cate_no=1019&display_group=1'),
 (5,
  'MISSHA Time Revolution Best Seller Special Set',
  54.99,
  46.74,
  'https://jolse.com/product/detail.html?product_no=42310&cate_no=1019&display_group=1'),
 (6,
  'Dr.Ceuracle Pro Balance Biotics Toner 300ml',
  30.0,
  25.5,
  'https://jolse.com/product/detail.html?product_no=42292&cate_no=1019&display_group=1'),
 (7,
  'Dr.Ceuracle Cica Regen 92 Toner 1

In [33]:
db.close()

## db 테이블에 저장된 데이터 df로 가져오기

In [34]:
db = conn('beauty_shop')


In [35]:
sql = "select * from product"
df = pd.read_sql(sql,db)
df

,PRODUCT_CODE,TITLE,ORI_PRICE,DISCOUNT_PRICE,link
0,1,B_LAB Matcha Hydrating Facial Toner 150ml,17.00,11.90,https://jolse.com/product/detail.html?product_...
1,2,META FORET 12PM Calming Toner 150ml,13.20,11.22,https://jolse.com/product/detail.html?product_...
2,3,LOOKS&MEII Rich Routine Balance Toner 150ml,21.00,17.85,https://jolse.com/product/detail.html?product_...
3,4,LOOKS&MEII Fresh Routine Balance Toner 150ml,21.00,17.85,https://jolse.com/product/detail.html?product_...
4,5,MISSHA Time Revolution Best Seller Special Set,54.99,46.74,https://jolse.com/product/detail.html?product_...
...,...,...,...,...,...
518,519,It's skin Aloe Relaxing Toner 150ml,19.00,16.15,https://jolse.com/product/detail.html?product_...
519,520,It's skin Hyaluronic Acid Moisture Toner 150ml,19.00,16.15,https://jolse.com/product/detail.html?product_...
520,521,Ciracle Base Toner pH5.6 105.5ml,23.48,19.96,https://jolse.com/product/detail.html?product_...
521,522,[Ciracle] Anti-Blemish Toner 105.5ml,22.25,18.91,https://jolse.com/product/detail.html?product_...


In [46]:
sql = "select title,ori_price,discount_price from product where discount_price > 15.00"
df = pd.read_sql(sql,db)
df

,title,ori_price,discount_price
0,SON&PARK; BEAUTY WATER 340ml,29.40,24.99
1,SOME BY MI AHA BHA PHA 30 Days Miracle Toner 1...,24.00,20.40
2,SOME BY MI Galactomyces Pure Vitamin C Glow To...,34.00,28.90
3,PURITO Centella Green Level Calming Toner 200ml,25.20,16.99
4,secretKey Hyaluron Aqua Soft Toner 500ml,22.00,18.70
...,...,...,...
448,It's skin Aloe Relaxing Toner 150ml,19.00,16.15
449,It's skin Hyaluronic Acid Moisture Toner 150ml,19.00,16.15
450,Ciracle Base Toner pH5.6 105.5ml,23.48,19.96
451,[Ciracle] Anti-Blemish Toner 105.5ml,22.25,18.91
